In [23]:
import os
import csv
import pandas as pd

import numpy as np
import gurobipy as gp
from gurobipy import GRB
from design_element_parameters import design_element_parameters
from EAC_lamp import EAC_lamp
from annuity_factor import annuity_factor
from fixed_costs import fixed_costs

In [31]:
r = 0.061

weights = {'investor': {'econ': 0.86, 'env':0.14},
          'policymaker': {'econ':0.7, 'env':0.3}}

location = 'Pingliang' # 'Jinshan'  'Langfang'  'Pingliang'  'Weifang'
 
tomato_dP = -0.3
gas_dP = 0
elec_dp = 0

# tomato_dP = 0
# gas_dP = 0.2
# elec_dp = 0.2

path = '../../Results analysis/Price scenarios/{}_tomato_dP={}_gas_dP={}_elec_dP={}.csv'.format(location, tomato_dP, gas_dP,elec_dp)

# path = '{}/{} all scores.csv'.format(location, location)
# df =  pd.read_csv(path).drop_duplicates()[['string', 'revenue', 'GHG', 'variable_costs', 'operational_income' ,'elec']]
df =  pd.read_csv(path).drop_duplicates()[['string', 'updated_revenue', 'GHG', 'updated_variable_costs', 'updated_operational_income' ,'elec']]
df = df.rename(columns = {'updated_revenue':'revenue',
                     'updated_variable_costs':'variable_costs',
                     'updated_operational_income':'operational_income'})
print(len(df))
df.head()

11781


,string,revenue,GHG,variable_costs,operational_income,elec
0,DCABEDGCB,270.682393,100.186009,190.493712,-8.912433,105.19
1,BAGACAHDB,414.147127,147.220113,248.811900,85.625805,173.59
2,AAABBCEEA,521.046741,171.906654,263.071513,172.521384,247.19
3,BBEACDBEA,243.778740,103.894171,196.466723,-18.920120,119.22
4,BBACDDCAA,257.445268,135.453711,193.933544,-11.460417,186.93


In [32]:
def df_2_raw_data(df):
    """Read csv as raw data
       Out: dict of dict, key1: index, key2: ['string','income','GHG','elec']
    """
    data = {}
    for i in range(len(df)):
        data[i] = {'string':df.iloc[i]['string'],
                   'income':df.iloc[i]['operational_income'],
                   'revenue':df.iloc[i]['revenue'],
                   'variable_costs':df.iloc[i]['variable_costs'],
                   'GHG':df.iloc[i]['GHG'],
                   'elec': df.iloc[i]['elec']}
    return data

In [33]:
def dea_data(data):
    """Process raw data (output of df_2_raw_data(df)) into gurobi.multidict
       Return: tuple of three elements:
            - list of strings,
            - dict of dict, inputs, key1: string, key2:['fixed_costs', 'variable_costs']
            - dict of dict, inputs, key1: string, key2:['revenue', 'income', 'GHG']
    """
    dea_dict = {}
    for i in range(len(data)):
        string = data[i]['string']
        elec = data[i]['elec']
        try:
            fixed_costs_ = fixed_costs(string, r, elec)
        except TypeError:
            print(i, string, r, elec)
        dea_dict[data[i]['string']] = [{'sum_EAC':fixed_costs_, 'variable_costs':data[i]['variable_costs']},\
                             {'revenue':data[i]['revenue'], 'income':data[i]['income'], 'GHG':data[i]['GHG']}]
    
    return gp.multidict(dea_dict)

In [34]:
data = df_2_raw_data(df)

In [35]:
dmus, inputs, outputs  = dea_data(data)

In [36]:
input_elements = ['sum_EAC', 'variable_costs']

def DEA_SUM_k(k, dmus, inputs, outputs, w_econ, w_env):
    """k: index, the kth observation
       data: raw data, output of df_2_raw_data()
       w_econ, w_env: float, relative importance of economic and environmental outcomes, sum up to 1 
       directional vector = (x, y, E;(0, w_econ*Y0, -w_env*E0))
       Return: beta (inefficiency) for DMU j
       NOTE: instead of using single EAC as input, this version uses sum of EACs as one input (fixed costs)
    """
    
    # Initialize Gurobi model 
    model = gp.Model('DEA')
    
        
    # Decision variables
    weights =  model.addVars(dmus, lb=0.0, name="Weight")
    beta = model.addVar(name="beta")
    
    # Constraints
    # sum of weights = 1
    C1 = model.addConstr(weights.sum('*')==1)
    # weighted EAC_sum of all DUMs <= EAC_sum of DMU j
    C2 = model.addConstr(gp.quicksum(weights[j]*inputs[j]['sum_EAC'] for j in dmus) <= inputs[dmus[k]]['sum_EAC'])
    # weighted variable costs i of all DUMs <= input i of DMU j
    C3 = model.addConstr(gp.quicksum(weights[j]*inputs[j]['variable_costs'] for j in dmus) <= inputs[dmus[k]]['variable_costs'])
    # weighted sum of revenue >= (1+beta)*revenue of DMU j
    C4 = model.addConstr(gp.quicksum(weights[j]*outputs[j]['revenue'] for j in dmus) >= (1+beta*w_econ)*outputs[dmus[k]]['revenue'])
    # weighted sum of GHG <= (1-beta)*GHG of DMU j
    C5 = model.addConstr(gp.quicksum(weights[j]*outputs[j]['GHG'] for j in dmus) <= (1-beta*w_env)*outputs[dmus[k]]['GHG'])
    
    # Objective function
    model.setObjective(beta, GRB.MAXIMIZE)
    
    model.Params.LogToConsole = 0
    model.optimize()
#     model.write("file.lp")
    
        
    
    return dmus[k], beta.x
    

In [ ]:
df_top_10 = df.sort_values(by=['operational_income'],ascending=False).head(10).round(1)
string_top_10 = list(df_top_10['string'])

role = 'investor'
# role = 'policymaker'

for string in string_top_10: 
    k = dmus.index(string)
    results = DEA_SUM_k(k, dmus, inputs, outputs, weights[role]['econ'], weights[role]['env'])
    print(k, results)

In [40]:
df_descending = df.sort_values(by=['operational_income'],ascending=False).round(1)
string_descending = list(df_descending['string'])

fieldnames = ['rank', 'string', 'investor', 'policymaker', 'EAC_sum','variable_costs', 'revenue', 'income', 'GHG']

# path = '../../Results analysis/DEA/new DDF EAC_sum {}_baseline.csv'.format(location)
path = '../../Results analysis/DEA/new DDF {}_tomato_dP={}_gas_dP={}_elec_dP={}.csv'.format(location, tomato_dP, gas_dP,elec_dp)

for rank, string in enumerate(string_descending[4278:]): 
    rank = rank+4278
    
    k = dmus.index(string)
    investor_results = DEA_SUM_k(k, dmus, inputs, outputs, weights['investor']['econ'], weights['investor']['env'])[-1]
    policymaker_results = DEA_SUM_k(k, dmus, inputs, outputs, weights['policymaker']['econ'], weights['policymaker']['env'])[-1]
    
    EAC_sum = inputs[string]['sum_EAC']
    variable_costs = inputs[string]['variable_costs']
    revenue = outputs[string]['revenue']
    income = outputs[string]['income']
    GHG = outputs[string]['GHG']
    
    with open(path,'a', newline='') as f:
        
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writerow({'rank':rank, 'string':string, 'investor':investor_results, 'policymaker': policymaker_results,\
                         'EAC_sum':EAC_sum,'variable_costs':variable_costs, 'revenue':revenue, 'income':income, 'GHG':GHG})
        
    if investor_results == 0 or policymaker_results == 0:
        print(rank, string, 'investor',investor_results, 'policymaker', policymaker_results)
        

7480 CAABAAICA investor 0.0 policymaker 0.0
8554 CAAAAAEEB investor 0.0 policymaker 0.0
9566 CAABAAHEA investor 0.0 policymaker 0.0
10133 CBBABAAAA investor 0.0 policymaker 0.0
10136 CBAABAADA investor 0.0 policymaker 0.0
10143 CAAABAADA investor 0.0 policymaker 0.0
10172 CBACEAADB investor 0.0 policymaker 0.0
10180 CBBAEAACA investor 0.0 policymaker 0.0
10202 CBAAEAADB investor 0.0 policymaker 0.0
10214 CBCAEAAEA investor 0.0 policymaker 0.0
10320 CBABEBAEA investor 0.0 policymaker 0.0
10334 CBCBECAAA investor 0.0 policymaker 0.0
11613 CAAAAAAEB investor 0.0 policymaker 0.0


In [54]:
locations = [ 'Pingliang']

scenarios = ['baseline',
             [-0.3, 0, 0], 
             [0, 0.2, 0.2]]

starting_k = {'Pingliang':{'baseline':2154 , 'LT':2340, 'HE':2360}, 
              'Weifang':{'baseline':0, 'LT':183, 'HE':148}}

for location in locations:
    for scenario in scenarios:
        if scenario == 'baseline':
            path = '{}/{} all scores.csv'.format(location, location)
            df =  pd.read_csv(path).drop_duplicates()[['string', 'revenue', 'GHG', 'variable_costs', 'operational_income' ,'elec']]
            DDF_path = '../../Results analysis/DEA/DDF EAC_sum {}_baseline.csv'.format(location)
            IND = starting_k[location]['baseline']
        else:
            tomato_dP = scenario[0]
            gas_dP = scenario[1]
            elec_dp = scenario[2]
            path = '../../Results analysis/Price scenarios/{}_tomato_dP={}_gas_dP={}_elec_dP={}.csv'.format(location, tomato_dP, gas_dP,elec_dp)
            df =  pd.read_csv(path).drop_duplicates()[['string', 'updated_revenue', 'GHG', 'updated_variable_costs', 'updated_operational_income' ,'elec']]
            df = df.rename(columns = {'updated_revenue':'revenue',
                     'updated_variable_costs':'variable_costs',
                     'updated_operational_income':'operational_income'})
            
            DDF_path = '../../Results analysis/DEA/DDF {}_tomato_dP={}_gas_dP={}_elec_dP={}.csv'.format(location, tomato_dP, gas_dP,elec_dp)
        
            if scenario[0] != 0:
                IND = starting_k[location]['LT']
            if scenario[0] == 0:
                IND = starting_k[location]['HE']
                
        data = df_2_raw_data(df)
        dmus, inputs, outputs  = dea_data(data)
        
        df_descending = df.sort_values(by=['operational_income'],ascending=False).round(1)
        string_descending = list(df_descending['string'])

        fieldnames = ['rank', 'string', 'investor', 'policymaker', 'EAC_sum','variable_costs', 'revenue', 'income', 'GHG']

        
        for rank, string in enumerate(string_descending[IND:]): 
            rank = rank+IND
    
            k = dmus.index(string)
            investor_results = DEA_SUM_k(k, dmus, inputs, outputs, weights['investor']['econ'], weights['investor']['env'])[-1]
            policymaker_results = DEA_SUM_k(k, dmus, inputs, outputs, weights['policymaker']['econ'], weights['policymaker']['env'])[-1]
    
            EAC_sum = inputs[string]['sum_EAC']
            variable_costs = inputs[string]['variable_costs']
            revenue = outputs[string]['revenue']
            income = outputs[string]['income']
            GHG = outputs[string]['GHG']
    
            with open(DDF_path,'a', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writerow({'rank':rank, 'string':string, 'investor':investor_results, 'policymaker': policymaker_results,\
                         'EAC_sum':EAC_sum,'variable_costs':variable_costs, 'revenue':revenue, 'income':income, 'GHG':GHG})

In [ ]:
string_descending[1667]

In [ ]:
string = 'DCCADBGDA'
print(outputs[string])
print(inputs[string])

In [ ]:
string_descending.index('DCCADBGDA')

In [ ]:
print('investor', round(DEA_SUM_k(10922, dmus, inputs, outputs, weights['investor']['econ'], weights['investor']['env'])[-1],3))

#### Inefficiency score of a single design

In [ ]:
# ================= the effect of lighting ===============

# string = 'CABABAAAA' # single PE, no light, no CO2
# string = 'CABABACAA' # single PE, HPS, 100 μmol m−2 s−1, no CO2
# string = 'CABABAEAA' # single PE, HPS, 200 μmol m−2 s−1, no CO2
# string = 'CABABAGAA' # single PE, LED, 100 μmol m−2 s−1, no CO2
# string = 'CABABAIAA' # single P"E, LED, 200 μmol m−2 s−1, no CO2

# string = 'CBBABAAAA' # double PE, no light, no CO2
# string = 'CBBABACAA' # double PE, HPS, 100 μmol m−2 s−1, no CO2
# string = 'CBBABAEAA' # double PE, HPS, 200 μmol m−2 s−1, no CO2
# string = 'CBBABAGAA' # double PE, LED, 100 μmol m−2 s−1, no CO2
# string = 'CBBABAIAA' # double PE, LED, 200 μmol m−2 s−1, no CO2
# string = 'CBBABAIAA' # double PE, LED, 150 μmol m−2 s−1, no CO2

# string = 'FCBABAAAA' # Venlo, no light, no CO2
# string = 'FCBABACAA' # Venlo, HPS, 100 μmol m−2 s−1, no CO2
# string = 'FCBABAEAA' # Venlo, HPS, 200 μmol m−2 s−1, no CO2
# string = 'FCBABAGAA' # Venlo, LED, 100 μmol m−2 s−1, no CO2
# string = 'FCBABAIAA' # Venlo, LED, 200 μmol m−2 s−1, no CO2

# ================= the effect of CO2 ================
# string = 'CABABAAAA' # single PE, no light， CO2=0
# string = 'CABABAACA' # single PE, no light， CO2=100
# string = 'CABABAAEA' # single PE, no light， CO2=200

# string = 'CABABAGAA' # single PE, LED 100， CO2=0
# string = 'CABABAGCA' # single PE, LED 100， CO2=100
# string = 'CABABAGEA' # single PE, LED 100， CO2=200

# string = 'CABABAIAA' # single PE, LED 200， CO2=0
# string = 'CABABAICA' # single PE, LED 200， CO2=100
# string = 'CABABAIEA' # single PE, LED 200， CO2=200

# string = 'FCBABAAAA' # Venlo, no light， CO2=0
# string = 'FCBABAACA' # Venlo, no light， CO2=100
# string = 'FCBABAAEA' # Venlo, no light， CO2=200

# string = 'FCBABAGAA' # Venlo, LED 100， CO2=0
# string = 'FCBABAGCA' # Venlo, LED 100， CO2=100
# string = 'FCBABAGEA' # Venlo, LED 100， CO2=200

# string = 'FCBABAIAA' # Venlo, LED 200， CO2=0
# string = 'FCBABAICA' # Venlo, LED 200， CO2=100
# string = 'FCBABAIEA' # Venlo, LED 200， CO2=200

# light_option = {'no light':'A', 'HPS 100':'C','LED 100':'G', 'HPS 200':'E', 'LED 200':'I'}
# for light in light_option.keys():
#     print(light)
#     string = f'FCBABA{light_option[light]}AA'
#     index = dmus.index(string)
#     print('investor', round(DEA_SUM_k(index, dmus, inputs, outputs, weights['investor']['econ'], weights['investor']['env'])[-1],3))
#     print('policymaker', round(DEA_SUM_k(index, dmus, inputs, outputs, weights['policymaker']['econ'], weights['policymaker']['env'])[-1],3))
#     print(outputs[string])
#     print(inputs[string],'\n')

CO2_option = {'no CO2':'A', 'CO2=100':'C','CO2=200':'E'}
for level in CO2_option.keys():
    print(level)
    string = f'CABABAI{CO2_option[level]}A'
    index = dmus.index(string)
    print('investor', round(DEA_SUM_k(index, dmus, inputs, outputs, weights['investor']['econ'], weights['investor']['env'])[-1],3))
    print('policymaker', round(DEA_SUM_k(index, dmus, inputs, outputs, weights['policymaker']['econ'], weights['policymaker']['env'])[-1],3))
    print(outputs[string])
    print(inputs[string],'\n')

# string = 'CABABAACA'
# role = 'investor'
# # role = 'policymaker'
# index = dmus.index(string)

# print('investor', round(DEA_SUM_k(index, dmus, inputs, outputs, weights['investor']['econ'], weights['investor']['env'])[-1],3))
# print('policymaker', round(DEA_SUM_k(index, dmus, inputs, outputs, weights['policymaker']['econ'], weights['policymaker']['env'])[-1],3))
# print(outputs[string])
# print(inputs[string])